# Appendix

## Code Reference

In [17]:
import sys
sys.path.insert(1, '../src')

from app import Main
from stock_data import StockData
import inspect # standard library used later to get info about the source code

def print_code(code): # prints '{line} {code}' with 2 less indent and without the def header
    codeline = lambda code, start : [(start + 1 + i, code[i]) for i in range(len(code))]
    print("".join([f"{line} {text[2:]}" if len(text) > 1 else f"{line} {text}" for line, text in codeline(code[0][1:], code[1])]))

### `app.py`

#### __ init __(self)

In [18]:
print_code(inspect.getsourcelines(Main.__init__))

59 """
60 initializes and sets up GUI widgets and its connections
61 """
62 super().__init__()
63 self.setupUi(self)
64 self.setWindowTitle("Stock Chart & Moving Average Application")
65 
66 # sets up figure to plot on, instantiates canvas and toolbar
67 self.figure, self.ax = plt.subplots()
68 self.canvas = FigureCanvas(self.figure)
69 self.toolbar = NavigationToolbar(self.canvas, self)
70 
71 # attaches the toolbar and canvas to the canvas layout
72 self.canvasLayout.addWidget(self.toolbar)
73 self.canvasLayout.addWidget(self.canvas)
74 
75 # sets up a scroll area to display GUI statuses
76 self.scrollWidget = qtw.QWidget()
77 self.scrollLayout = qtw.QVBoxLayout()
78 self.scrollWidget.setLayout(self.scrollLayout)
79 self.scrollArea.setWidget(self.scrollWidget)
80 
81 # button & checkbox connections
82 self.loadCSVButton.clicked.connect(self.load_data)
83 self.updateWindowButton.clicked.connect(self.update_canvas)
84 self.SMA1Checkbox.stateChanged.connect(self.update_canvas)
85 self.S

#### `load_data(self)`

In [19]:
print_code(inspect.getsourcelines(Main.load_data))

91 """
92 loads stock data .csv from inputted filepath string on the GUI
93 as StockData object, also autocompletes all inputs
94 using information provided by the csv.
95 
96 Error handling
97 	invalid filepath :
98 		empty filepath or file could not be found.
99 	invalid .csv :
100 		.csv file is empty, missing date column, etc.
101 """
102 filepath = Path(self.filePathEdit.text())
103 
104 try:
105 	self.stock_data = StockData(filepath)
106 	start_date, end_date = self.stock_data.get_period()
107 	period = f"{start_date} to {end_date}"
108 
109 	# auto-complete feauture
110 	self.startDateEdit.setText(start_date)
111 	self.endDateEdit.setText(end_date)
112 	self.periodEdit.setText(period)
113 	self.SMA1Edit.setText("15")
114 	self.SMA2Edit.setText("50")
115 	self.SMA1Checkbox.setChecked(False)
116 	self.SMA2Checkbox.setChecked(False)
117 
118 	self.report(f"Data loaded from {filepath}; period auto-selected: {start_date} to {end_date}.")
119 	print(self.stock_data.data)
120 
121 exce

#### `update_canvas(self)`

In [20]:
print_code(inspect.getsourcelines(Main.update_canvas))

128 """
129 creates a datetime object from the inputted date string
130 of format YYYY-MM-DD. uses it to slice a copy of loaded
131 stock_data to be used to update graphics. checks
132 checkboxes first to see if SMA1, SMA2, Buy and Sell plots
133 need to be drawn. finally, updates graphic accordingly.
134 
135 Error handling
136 invalid date format:
137 	date format inside the .csv file is not YYYY-MM-DD
138 non-existent stock_data :
139 	the selected range results in an empty dataframe
140 	or end date < start date
141 non-existent data point :
142 	data of that date does not exist,
143 	or maybe because it is Out-Of-Bound
144 raised exceptions :
145 	SMA1 and SMA2 values are the same,
146 	or other exceptions raised
147 """
148 self.date_format = '%Y-%m-%d'
149 
150 try:
151 	start_date = str(datetime.strptime(self.startDateEdit.text(), self.date_format).date())
152 	end_date = str(datetime.strptime(self.endDateEdit.text(), self.date_format).date())
153 	period = f"{start_date} to {e

#### `plot_graph(self, column_headers, formats)`

In [21]:
print_code(inspect.getsourcelines(Main.plot_graph))

194 """
195 plots graphs specified under columnd_headers using the formats
196 
197 Parameters
198 column_headers : [str, str, ...]
199 	a list containing column header names with data to be plotted
200 formats : [str, str, ...]
201 	a list of matplotlib built-in style strings to indicate
202 	whether to plot line or scatterplot and the colours
203 	corresponding to each value in col_headers
204 	(hence, must be same length)
205 
206 Error handling
207 empty dataframe :
208 	selected dataframe is empty
209 """
210 self.ax.clear()
211 assert not self.selected_stock_data.empty
212 
213 # matplotlib has its own internal representation of datetime
214 # date2num converts datetime.datetime to this internal representation
215 x_data = list(mdates.date2num(
216                               [datetime.strptime(dates, self.date_format).date()
217                               for dates in self.selected_stock_data.index.values]
218                               ))
219 
220 colors = ['black', 'bl

#### `report(self, string)`

In [22]:
print_code(inspect.getsourcelines(Main.report))

242 """
243 given a report (string), update the scroll area with this report
244 
245 Parameters
246 string : str
247 	string of the report, usually the error message itself.
248 """
249 report_text = qtw.QLabel(string)
250 self.scrollLayout.addWidget(report_text)
251 print(string)



#### `center(self)`

In [23]:
print_code(inspect.getsourcelines(Main.center))

254 """
255 centers the fixed main window size according to user screen size
256 """
257 screen = qtw.QDesktopWidget().screenGeometry()
258 main_window = self.geometry()
259 x = (screen.width() - main_window.width()) / 2
260 
261 # pulls the window up slightly (arbitrary)
262 y = (screen.height() - main_window.height()) / 2 - 50
263 self.setFixedSize(main_window.width(), main_window.height())
264 self.move(x, y)



### `stock_data.py`

#### __ init __(self)

In [24]:
print_code(inspect.getsourcelines(StockData.__init__))

18 """
19 initializes StockData object by parsing stock data .csv file into a dataframe
20 (assumes 'Date' column exists and uses it for index),
21 also checks and handles missing data
22 
23 Parameters
24 filepath : str
25 	filepath to the stock data .csv file, can be relative or absolute
26 
27 Raises
28 IOError :
29 	failed I/O operation, e.g: invalid filepath, fail to open .csv
30 """
31 self.filepath = filepath
32 self.data = pd.read_csv(filepath).set_index('Date')
33 self.check_data()



#### check_data(self, overwrite=True)

In [25]:
print_code(inspect.getsourcelines(StockData.check_data))

36 """
37 checks and handles missing data by filling in missing values by interpolation
38 
39 Parameters
40 overwrite : bool (True)
41 	if True, overwrites original source stock data .csv file
42 
43 Returns
44 self : StockData
45 """
46 # function to fill in missing values
47 # by averaging previous data and after (interpolation)
48 self.data = self.data.interpolate()
49 self.data.to_csv(self.filepath, index=overwrite)
50 return self



#### get_data(self, start_date, end_date)

In [26]:
print_code(inspect.getsourcelines(StockData.get_data))

53 """
54 returns a subset of the stock data from start_date to end_date inclusive
55 
56 Parameters
57 start_date : str
58 	start date of stock data range, must be of format YYYY-MM-DD
59 end_date : str
60 	end date of stokc data range, must be of format YYYY-MM-DD
61 
62 Returns:
63 selected_data : DataFrame
64 	stock data dataframe indexed from specified start to end date inclusive
65 
66 Raises
67 KeyError :
68 	data for this date does not exist
69 AssertionError :
70 	selected range is empty
71 """
72 self.selected_data = self.data[str(start_date):str(end_date)]
73 return self.selected_data



#### get_period(self)

In [27]:
print_code(inspect.getsourcelines(StockData.get_period))

76 """
77 returns a string tuple of the first and last index
78 which make up the maximum period of StockData
79 
80 Returns
81 period : (str, str)
82 
83 Raises
84 TypeError :
85 	the return tuple is probably (nan, nan) because .csv is empty
86 """
87 index = list(self.data.index)
88 (first, last) = (index[0], index[-1])
89 return (first, last)



#### _calculate_SMA(self, n, col='Close')

In [28]:
print_code(inspect.getsourcelines(StockData._calculate_SMA))

92 """
93 calculates simple moving average (SMA) and augments the stock dataframe
94 with this SMA(n) data as a new column
95 
96 Parameters
97 n : int
98 	the amount of stock data to use to calculate average
99 col : str ('Close')
100 	the column head title of the values to use to calculate average
101 
102 Returns
103 self : StockData
104 """
105 col_head = f'SMA{n}'
106 if col_head not in self.data.columns:
107 	sma = self.data[col].rolling(n).mean()
108 	self.data[f'SMA{n}'] = np.round(sma, 4)
109 	self.data.to_csv(self.filepath, index=True)
110 return self



#### _calculate_crossover(self, SMA1, SMA2, col='Close')

In [29]:
print_code(inspect.getsourcelines(StockData._calculate_crossover))

113 """
114 calculates the crossover positions and values,
115 augments the stock dataframe with 2 new columns
116 'Sell' and 'Buy' containing the value at which SMA crossover happens
117 
118 Parameters
119 SMA1 : str
120 	the first column head title containing the SMA values
121 SMA2 : str
122 	the second column head title containing the SMA values
123 col : str ('Close')
124 	the column head title whose values will copied into 'Buy' and 'Sell'
125 	columns to indicate crossovers had happen on that index
126 
127 Returns
128 self : StockData
129 
130 Raises
131 Exception :
132 	SMA1 and SMA2 provided are the same, they must be different
133 """
134 if SMA1 < SMA2: signal = self.data[SMA1] - self.data[SMA2]
135 elif SMA1 > SMA2: signal = self.data[SMA2] - self.data[SMA1]
136 else: raise Exception(f"{SMA1} & {SMA2} provided are the same. They must be different SMA.")
137 
138 signal[signal > 0] = 1
139 signal[signal <= 0] = 0
140 diff = signal.diff()
141 
142 self.data['Sell'] = np.nan

#### plot_graph(self, col_headers, style, ax, show=True)

In [30]:
print_code(inspect.getsourcelines(StockData.plot_graph))

151 """
152 plots columns of selected values as line plot and/or columns of values
153 as scatter plot as specified by style to an Axes object
154 
155 Parameters
156 col_headers : [str, str, ...]
157 	a list containing column header names whose data are to be plotted
158 style : [str, str, ...]
159 	a list of matplotlib built-in style strings to indicate whether to plot
160 	line or scatterplot and the colours corresponding to each value in
161 	col_headers (hence, must be same length)
162 ax : Axes
163 	matplotlib axes object on which the plot will be drawn
164 
165 Raises
166 AttributeError :
167 	self.selected_data has not been specified,
168 	call StockData.get_data(start, end) before plotting
169 AssertionError :
170 	self.selected_data is empty, perhaps due to OOB or invalid range
171 """
172 assert not self.selected_data.empty
173 self.selected_data[col_headers].plot(style=style,
174                                      ax=ax,
175                                      grid=True,

#### calculate_SMA(self, n)

In [31]:
print_code(inspect.getsourcelines(StockData.calculate_SMA))

181 """
182 calculates simple moving average (SMA) and augments the stock dataframe
183 with this SMA(n) data as a new column
184 
185 Parameters
186 n : int
187 	the amount of stock data to use to calculate average
188 col : str ('Close')
189 	the column head title of the values to use to calculate average
190 
191 Returns
192 self : StockData
193 """
194 col_head = 'SMA' + str(n)
195 df = self.data.reset_index()
196 
197 if col_head not in df.columns:
198 	# Extract full dataframe from the actual data
199 	# (to check if there is enough data for sma)
200 	dateList = self.data.index.values.tolist()
201 	returnList = []
202 	for date in dateList: # for date in dateList
203 	# find the index of date in the full data
204 		dateIndex = df[df["Date"]==date].index.values[0]
205 		if dateIndex < n: # if date index is less than n: append None
206 			returnList.append(np.nan)
207 		else:
208 			sum = 0
209 			for i in range(n):
210 				sum += df.iloc[dateIndex-i]["Close"]
211 			# append the S

#### calculate_crossover(self, SMAa, SMAb)

In [32]:
print_code(inspect.getsourcelines(StockData.calculate_crossover))

221 """
222 calculates the crossover positions and values,
223 augments the stock dataframe with 2 new columns
224 'Sell' and 'Buy' containing the value at which SMA crossover happens
225 
226 Parameters
227 SMA1 : str
228 	the first column head title containing the SMA values
229 SMA2 : str
230 	the second column head title containing the SMA values
231 col : str ('Close')
232 	the column head title whose values will copied into 'Buy' and 'Sell'
233 	columns to indicate crossovers had happen on that index
234 
235 Returns
236 self : StockData
237 
238 Raises
239 Exception :
240 	SMA1 and SMA2 provided are the same, they must be different
241 """
242 col_head1 = 'Position'
243 col_head2 = 'Signal'
244 col_head3 = 'Buy'
245 col_head4 = 'Sell'
246 df = self.data
247 
248 # to ensure the correct number of elements in the loop
249 SMAlist = self.data.index.values.tolist()
250 # extracts the SMA from the specific column in self.data
251 if SMAa < SMAb:
252 	SMA1 = df[SMAa].tolist()
253 	SMA